**Sommario**

- [Operazioni CRUD con Flask-SQLAlchemy](#operazioni-crud-con-flask-sqlalchemy)
  - [`models.py`](#modelspy)
  - [`app.py`](#apppy)
  - [C.R.U.D.](#crud)
    - [CREATE con `db.session.add()`](#create-con-dbsessionadd)
    - [READ con `Model.query`](#read-con-modelquery)
    - [UPDATE con attributi](#update-con-attributi)
    - [DELETE con `db.session.delete()`](#delete-con-dbsessiondelete)
  - [Gestione delle transazioni](#gestione-delle-transazioni)
    - [`db.session.commit()` per salvare definitivamente](#dbsessioncommit-per-salvare-definitivamente)
    - [`db.session.flush()` per salvare ogni tanto durante la sessione](#dbsessionflush-per-salvare-ogni-tanto-durante-la-sessione)
    - [`db.session.rollback()` per annullare manualmente le operazioni](#dbsessionrollback-per-annullare-manualmente-le-operazioni)
    - [Esempio di flush, rollback e commit](#esempio-di-flush-rollback-e-commit)

# Operazioni CRUD con Flask-SQLAlchemy

Come avete appreso nelle lezioni di SQL, l'acronimo [**CRUD**](https://it.wikipedia.org/wiki/CRUD) sta per CREATE, READ, UPDATE e DELETE, ovvero le quattro operazioni fondamentali che possiamo eseguire sui database.

Vediamo come effettuare queste operazioni con l'ORM di SQLAlchemy.

## `models.py`

Immaginiamo di avere una tabella rappresentata da una classe-modello come questo, nel `models.py`:

```python
# models.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()  # Crea l'istanza di SQLAlchemy

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    age = db.Column(db.Integer)
```

## `app.py`

Per poter svolgere le operazioni CRUD nel file del "controller" `app.py`, dobbiamo prima creare e inizializzare correttamente le istanze di Flask e SQLAlchemy e infine creare le tabelle nel database.

Dopodiché abbiamo due scelte:

- Effettuare le operazioni CRUD prima dell'avvio dell'applicazione Flask. Questo può essere utile per effettuare delle operazioni come l'inizializzazione e il popolamento delle tabelle, azioni di migrazione ecc.

- Effettuare le operazioni CRUD all'interno delle funzioni "endpoint" che gestiscono le route, dunque dopo che l'applicazione e il server Flask sono stati avviati.

```python
# app.py

from flask import Flask
from models import db, User

app = Flask(__name__)  # Crea l'istanza di Flask

app.config.update(
    SQLALCHEMY_DATABASE_URI='sqlite:///db.sqlite3',  # Path al file del DB
)

db.init_app(app)  # Inizializza l'istanza di SQLAlchemy con l'app Flask

...  # <--- QUA PUOI SVOLGERE LE OPERAZIONI CRUD
...  # <--- ALL'INTERNO DELLE FUNZIONI DI ENDPOINT
...  # <--- CHE GESTISCONO LE ROUTE.

if __name__ == '__main__':
    with app.app_context():
        db.create_all()  # Crea le tabelle nel database, se necessario

        ...  # <--- QUA PUOI SVOLGERE LE OPERAZIONI CRUD PRIMA
        ...  # <--- DELL'AVVIO DELL'APP FLASK. PUO' ESSERE UTILE 
        ...  # <--- PER INIZIALIZZARE E POPOLARE IL DB.

    app.run()

```

Vediamo come eseguire le quattro classiche operazioni CRUD con Flask-SQLAlchemy.

## C.R.U.D.

### CREATE con `db.session.add()`

Creare un nuovo record nel database:

```python
new_user = User(name='Alice', email='alice@flask.sql')
db.session.add(new_user)
db.session.commit()

new_user_id = new_user.id  # es. 10
```

### READ con `Model.query`

Per leggere i record dal database:

```python
the_user = User.query.get(10)
all_users = User.query.all()
some_users = User.query.filter(User.age > 18).order_by(User.name).all()
...
```

> PER APPROFONDIRE: Vedi il notebook [sqlalchemy_READ_legacy.ipynb](sqlalchemy_READ_legacy.ipynb)

### UPDATE con attributi

Per aggiornare un record esistente:

```python
user = User.query.get(10)
user.name = 'Bob'
user.email = 'bob@flask.sql'
db.session.commit()
```

### DELETE con `db.session.delete()`

Per eliminare un record esistente:

```python
user = User.query.get(10)
db.session.delete(user)
db.session.commit()
```

## Gestione delle transazioni

Una transazione è una sequenza di operazioni CRUD che vengono eseguite come un'unità logica di lavoro. Una transazione ha quattro proprietà fondamentali, note come [**ACID**](https://it.wikipedia.org/wiki/ACID):

1. **Atomicità**: Ogni transazione è "tutto o niente". O tutte le operazioni nella transazione vengono completate con successo, oppure nessuna di esse viene applicata.
2. **Consistenza**: Una transazione porta il database da uno stato consistente a un altro stato consistente.
3. **Isolamento**: Le transazioni parallele non interferiscono tra loro.
4. **Durabilità**: Una volta che una transazione è stata committata, i cambiamenti sono permanenti nel database, anche in caso di guasti.

Per approfondire, vedi le sezioni [**Transactions and Connection Management**](https://docs.sqlalchemy.org/en/20/orm/session_transaction.html) e [**Session API**](https://docs.sqlalchemy.org/en/20/orm/session_api.html) nella documentazione ufficiale.


### `db.session.commit()` per salvare definitivamente

Il _**commit**_ dice al database di eseguire e finalizzare tutte le operazioni che sono state accodate nella sessione corrente.

Ecco come funziona nel contesto delle transazioni:

- **Inizio della transazione**: Quando inizi a fare operazioni con `db.session` (ad esempio, `db.session.add()`) per aggiungere un nuovo record, queste operazioni sono accumulate nella sessione ma non vengono ancora applicate al database.

- **Commit della transazione**: Quando chiami `db.session.commit()`, SQLAlchemy invia tutte le operazioni accumulate al database come una singola transazione. Se tutte le operazioni vanno a buon fine, la transazione viene committata e le modifiche diventano permanenti.

- **Rollback in caso di errore durante il commit**: Se durante il commit della transazione si verifica un errore proveniente dal DB, SQLAlchemy esegue automaticamente un rollback, annullando tutte le operazioni fatte nella transazione. Questo mantiene il database in uno stato consistente.

> **ATTENZIONE:** Se si verifica un errore prima del commit, l'utente può decidere se eseguire un rollback manualmente. Solitamente questa è una buona prassi per assicurare un comportamento ACID.

### `db.session.flush()` per salvare ogni tanto durante la sessione

Il _**flush**_ invia tutte le operazioni in sospeso nella sessione al database, ma senza committarle. Serve per sincronizzare la sessione con il database, assicurandosi che le modifiche siano visibili alle successive query nella stessa transazione.


### `db.session.rollback()` per annullare manualmente le operazioni

Il _**rollback**_ annulla tutte le operazioni fatte nella sessione corrente, riportando il database allo stato prima dell'inizio della transazione. Anche le operazioni di flush vengono annullate.

### Esempio di flush, rollback e commit

Ecco un esempio che mostra come utilizzare `flush`, `rollback` e `commit` secondo le buone pratiche:

```python
def create_user(name, email):
    # Prova:
    try:
        new_user = User(name=, email=email)
        db.session.add(new_user)
        db.session.flush()  # Sincronizza la sessione con il database
        # Ora new_user.id è disponibile nell'ORM
        print(f'Nuovo User ID: {new_user.id}')

        # Esegui altre operazioni che potrebbero fallire
        another_user = User.query.get(1)
        another_user.email = 'new_email@example.com'
    # Ma in caso di errore:
    except Exception as err:
        db.session.rollback()  # Annulla tutte le operazioni, anche quelle flushate
        print(f'Si è verificato un errore: {err}')
    # Se non si sono verificati errori:   
    else:
        db.session.commit()  # Salva e conferma tutte le operazioni

# Uso della funzione
create_user('John Doe', 'john.doe@example.com')
```